# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome() # abre navegador


In [40]:
# pesquisar no google cotação dolar
# pegar a cotação do dolar
navegador.get('https://www.google.com.br/') # vai para google
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pega cotação
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)



4.1539515300000005


In [41]:
# pesquisar no google cotação euro
# pegar a cotação do euro
navegador.get('https://www.google.com.br/') # vai para google
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pega cotação
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

6.471828566


In [42]:
# entrar no site melhor câmbio
# pegar a cotação do ouro

site = 'https://www.melhorcambio.com/ouro-hoje'
navegador.get(site)
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)



304.29


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [43]:
import pandas as pd

tabela_produtos = pd.read_excel('Produtos.xlsx')
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [44]:
tabela_produtos.loc[tabela_produtos['Moeda']=='Dólar', 'Cotação'] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos['Moeda']=='Euro', 'Cotação'] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos['Moeda']=='Ouro', 'Cotação'] = float(cotacao_ouro)

tabela_produtos['Preço Base Reais'] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map('{:.2f}'.format)

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,4.153952,4153.909990,1.40,5815.47
1,Carro Renault,4500.00,Euro,6.471829,29123.228547,2.00,58246.46
2,Notebook Dell,899.99,Dólar,4.153952,3738.514837,1.70,6355.48
3,IPhone,799.00,Dólar,4.153952,3319.007272,1.70,5642.31
4,Carro Fiat,3000.00,Euro,6.471829,19415.485698,1.90,36889.42
5,Celular Xiaomi,480.48,Dólar,4.153952,1995.890631,2.00,3991.78
6,Joia 20g,20.00,Ouro,304.290000,6085.800000,1.15,6998.67


### Agora vamos exportar a nova base de preços atualizada

In [45]:
tabela_produtos.to_excel('Produtos Atualizado.xlsx', index=False)